# Intro to DataFrames
A `DataFrame` is similar to a `TimeArray`, but has no (meaningfully) ordered index. DataFrames are great for data analysis and statistics. Documentation is [here](http://juliadata.github.io/DataFrames.jl/stable).

In [ ]:
# Pkg.add("DataFrames")
using DataFrames
using TimeSeries

# import two functions from DataFrames which have analogues in TimeSeries
import DataFrames: head, tail

In [ ]:
using Plots
gr()

## Basics
Let's construct a `DataFrame` with three columns (called 'Z', 'date', 'ret') of different types:

In [ ]:
# Z: from 6 to 1 in descending order, 
# date: 6 consequtive days from today, 
# ret: random numbers
df = DataFrame(
    Z=collect(6:-1:1), 
    date=[Dates.today() + Dates.Day(p) for p in collect(1:6)], 
    ret=randn(6));

print(df)

## Accessors
are more or less the same as in TimeArrays, just querying columns is done differently.

In [ ]:
# select a column
print(df[:date])

In [ ]:
# select a row
print(df[4,:])

In [ ]:
# select some rows and some columns
print(df[4:6, [:Z, :ret]])

In [ ]:
# select those rows for which Z > 3
print(df[df[:Z] .> 3, :])

In [ ]:
# head, tail (`k` rows)
k = 2
println("\nhead of df:\n")
print(head(df, k))
println("\ntail of df:\n")
print(tail(df, k))

## Joins
Let's create another DataFrame sharing one columns name with `df` and merge it with the latter:

In [ ]:
# date: 5 consequtive days from today
df_2 = DataFrame(
    date=[Dates.today() + Dates.Day(p) for p in collect(3:7)], 
    ret=randn(5))

println("\nold df\n")
print(df)
println("\nnew df, with dates overlapping those in df\n")
print(df_2)

Join `df` and `df_2` on column `A` with outer join (all rows are kept from both dataframes):

In [ ]:
df_merged_o = join(df, df_2, on=:date, kind = :outer);

println("\nmerged with outer join\n")
print(df_merged_o)

Merging might screw up the order of rows, but you can sort rows using `sort!` as follows:

In [ ]:
sort!(df_merged_o, cols=[order(:date)])

Join `df` and `df_2` on column `A` with inner join (only the rows available in both dataframes are kept):

In [ ]:
df_merged_i = join(df, df_2, on=:date, kind = :inner);

println("\nmerged with inner join\n")
print(df_merged_i)

## A useful function

In [ ]:
# summary of dataframe
print(describe(df))

## I/O
to/from .csv can be easily done with package `CSV` which offers enhanced functionality over the stuff built-in into `DataFrames`. Documentation is [here](http://juliadata.github.io/CSV.jl/stable/).

In [ ]:
# Pkg.add("CSV")
using CSV

### Output
Let's write `df` to file **/Results/df.csv**:

In [ ]:
# filename
hangar = "Results/df.csv";

# output
CSV.write(hangar, df, delim=',');

### Input
Let's read `df` back in:

In [ ]:
# filename
hangar = "Results/df.csv";

# read in
df_fom_csv = CSV.read(hangar, delim=',');

println("\ndf\n")
print(df_fom_csv)

println("\nsummary of df\n")
print(describe(df_fom_csv))

## Conversion to `TimeArray`
is done with the familiar `TimeArray` constructor, but one has to conform to its rules: `dates` and `values` must be instances of `Array`!

In [ ]:
# convert everything to array first
dt = convert(Array, Array(df[:date]));
val = convert(Array, df[[:Z, :ret]]);

# construct TimeArray
tarr = TimeArray(dt, val, ["Z", "date"])
plot(tarr)

# Fetching data from web sources
## Example: from fred.stlouisfed.org
St. Louis Fed offers a free API which GitHub user **micahjsmith** used to write a set of simple Julia functions, gathered in `FredData` package (registered). Documentation is [here](https://github.com/micahjsmith/FredData.jl).

In [ ]:
Pkg.add("FredData")

# for fetching data from St.Louis Fed
using FredData

First, we need a connection to the St. Louis Fed server, which is established with an API key. This should work:

**15c304c4c4a493bcc70ac9d3f3dcb94b**

but you can register your own key on the [website](https://research.stlouisfed.org/docs/api/api_key.html).

In [ ]:
f = Fred("15c304c4c4a493bcc70ac9d3f3dcb94b");

Then, the command to download any series (let's take the daily effective federal funds rate, called `DFF`) is as follows:

In [ ]:
# fetch data
request = get_data(f, "DFF", observation_start="2016-01-01");

Object `request` is a container with a bunch of stuff. We are first and foremost interested in the data itself:

In [ ]:
# retrieve the dataframe
ffr = request.df;
plot(ffr[:date], ffr[:value])

## Example: from the website of Kenneth French
located under http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

The files are listed as .zip archives, each with a name. We will download the research factors at the daily frequency.

In [ ]:
Pkg.add("ZipFile")

# For unpacking files from Kenneth French's website
using ZipFile

In [ ]:
# address and file name (mind the different web address!)
http = "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/";
filename = "F-F_Research_Data_Factors_daily_CSV.zip";

In [ ]:
# download
download(string(http, filename), string("Results/", filename));

Carefully inspect the contents of the .csv file you are about to parse. Mind the number of useless lines at the beginning and end, the number of columns and the date format -- neglecting any of these parameters will result in an error.

In [ ]:
# delimiter is comma ','
delim = ',';
# number of the line with column names
header = 5;
# number of columns with numerical data
N = 4;
# data types, starting with date
types = vcat(Date, fill(Float64, N));
# format of dates in the first columns
dateformat = "yyyymmdd";
# number of rows to read in
rows = 1000;

Now, unzip the archive and parse the single .csv file from it

In [ ]:
# unzip
unzipped = ZipFile.Reader(string("Results/", filename));

# parse
ff_factors = CSV.read(unzipped.files[1], 
    delim=delim, 
    header=header, 
    types=types, 
    dateformat=dateformat, 
    rows=rows);

# date column has no name -> rename it
rename!(ff_factors, Symbol(""), :date)

println("\nparsed data frame:\n")
print(ff_factors)

# Example: from Quandl

[Quandl](https://www.quandl.com/) offers a free API which GitHub user **milktrader** used to write `Quandl` package (registered). Documentation is [here](http://quandljl.readthedocs.io/en/latest/).

In [ ]:
Pkg.add("Quandl")
using Quandl

Once again, you will need an API key. This should work:

**xyPFrsWutTCZEpTWZAHg**

but you can always register an own key.

In [ ]:
# set key
set_auth_token("xyPFrsWutTCZEpTWZAHg")

Request series in the form of a DataFrame

In [ ]:
# request Bitcoin price over 2016 (from anothe exchange)
btc = quandl("BITFINEX/BTCUSD", from="2016-01-01", to="2016-12-31", format="DataFrame");

In [ ]:
# plot
plot(btc[:Date], btc[:Last], lw=1.5)